# Class and Magic Methods
---
## Class
### private and protected

In [1]:
class Base:
    def __init__(self, public, protected, private):
        self.public = public
        self._protected = protected
        self.__private = private
        print("Access attributes inside the class")
        print(f"public variable: {self.public}")
        print(f"protected variable: {self._protected}")
        print(f"private variable: {self.__private}")

base = Base('public variable', 'protected variable', 'private variable')
print(base.public)
print(base._protected)

# access a private variable from outside its class will result in an AttributeError
# print(base.__private) 

print(base._Base__private)

### property() function

```
Signatures:
prop = property(getter, setter, deleter, docstring)
```

In [2]:
class Person:
    def __init__(self, name='Anonymous'):
        self.__name = name

    def setname(self, name):
        self.__name = name
        
    def getname(self):
        return self.__name
    
p1 = Person()
print(f"default name: {p1.getname()}")
p1.setname('John')
print(f"new name: {p1.getname()}")

class PersonWithProperty:
    """
    property(getname, setname) returns the property object and assigns it to name.
    The name property hides the private instace attribute __name. The name property is
    accessed directly, but internally it will invoke the getname() or setname() method.
    """
    
    def __init__(self):
        self.__name = ''
        
    def setname(self, name):
        print('setname() called')
        self.__name = name
    
    def getname(self):
        print('getname() called')
        return self.__name
    
    def delname(self):
        print('delname() called')
        del self.__name
    
    name = property(getname, setname, delname)

p2 = PersonWithProperty()
p2.name = "Jane"
print(f"new name: {p2.name}")
del p2.name

### @property decorator

A decorator is a function that receives another function as argument. The behavior of the argument function is extended by the decorator without actually modifying it.

@property decorator is a built-in decorator in Python for the property() function.

### @classmethod decorator and @staticmethod decorator

- The @classmethod decorator can be applied on any method of a class. 
- The @staticmethod is a built-in decorator that defines a static method in the class. A static method doesn't receive any reference argument whether it is called by an instance of a class or by the class itself. -> doesn't have any arguments - neither `self` nor `cls` 

In [3]:
def mydecoratorfunction(some_function): # function to be decorated passed as argument
    def wrapper_function(): # wrap the some_function and extends its behaviour
        # write code to extend the behaviour of some_function()
        some_function() # call some_function
        return wrapper_function # return wrapper function

def display(string):
    print(string)
    
def displaydecorator(fn):
    def display_wrapper(string):
        print('Output:', end=" ")
        fn(string)
    return display_wrapper

out = displaydecorator(display)
out('Hello World')

@displaydecorator
def display2(string):
    print(f'{string} via @displaydecorator')
    
display2('Hello World')


class PersonWithPropertyDecorator:
    total_objects = 0

    def __new__(cls):
        cls.total_objects += 1
        instance = object.__new__(cls)
        return instance

    def __init__(self):
        self.__name = ''
        # an alternative for not using __new__ magic method
        # PersonWithPropertyDecorator.total_objects += 1
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        self.__name = name
        
    @name.deleter
    def name(self):
        print('Deleting..')
        del self.__name
        PersonWithPropertyDecorator.total_objects -= 1
        
    @classmethod
    def show_count(cls):
        print(f"Total persons: {cls.total_objects}")

    @staticmethod
    def greet():
        print('Hola!')
        
p3 = PersonWithPropertyDecorator()
p3.name = 'Steve'
print(f"get name {p3.name}")
PersonWithPropertyDecorator.show_count() # classmethod can be called using the class name
del p3.name
# p3.name # An AttributeError
p3.show_count() # classmethod can also be called using the object
p4 = PersonWithPropertyDecorator()
p4.show_count()
p5 = PersonWithPropertyDecorator()
p5.show_count()
PersonWithPropertyDecorator.greet() # staticmethod can be called using both the class name and the object
p5.greet()

# Python Magic Methods 

In [4]:
class Base:
    pass

dir(Base)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

## Initialization and Construction
### `__new__() method`

In Python the `__new__()` magic method is implicitly called before the `__init__()` method. The `__new__()` method returns a new object, which is then initialized by `__init__()`.

### `__init__() method`

To get called by the __new__ method. The constructor is always called when an object is created.
- default constructor: The default constructor is simple constructor which doesn't accept any arguments.
- parameterized constructor: constructor with parameters is known as parameterized constructor.

### `__del__() method`

Destructor method. The destructor was called **after the program ended** or when all the referenes to object are deleted i.e. when the reference count becomes zero, not when object went out of scope. It can be quite useful for objects that might require extra cleanup upon deletion, like `sockets` or `file` objects.

In [5]:
class Base:
    def __new__(cls):
        print("__new__ magic method is called")
        instance = object.__new__(cls)
        return instance
    
    def __init__(self):
        print("__init__ magic method is called")

Base()

__new__ magic method is called
__init__ magic method is called


## Comparison magic methods
### __cmp__(self, other)
- The most basic of the comparison magic methods.
- `__cmp__` should return a negative integer if `self < other`
- `__cmp__` should return zero if `self == other`
- `__cmp__` should return a positive integer if `self > other`
- It's usually best to define each comparison rather than define them all at once

### `__eq__(self, other)`
- Defines behavior for the equality operator, `==`

### `__ne__(self, other)`
- Defines behavior for the inequality operator, `!=`

### `__lt__(self, other)`
- Defines behavior for the less-than operator, `<`

### `__gt__(self, other)`
- Defines behavior for the greater-than operator, `>`

### `__le__(self, other)`
- Defines behavior for the less-than-or-equal-to operator, `<=`

### `__ge__(self, other)`
- Defines behavior for the greater-than-or-equal-to operator, `>=`

In [6]:
import heapq


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Wrapper:
    def __init__(self, node):
        self.node = node

    def __lt__(self, other):
        return self.node.val < other.node.val
    
    def __le__(self, other):
        return self.node.val <= other.node.val


class ListGenerator:
    def __init__(self):
        self.__root = ListNode()
        
    @property
    def root(self):
        return self.__root.next
    
    @root.setter
    def root(self, elems):
        ptr = self.__root
        for elem in elems:
            ptr.next = ListNode(elem)
            ptr = ptr.next

    @staticmethod
    def print_element(l):
        ptr = l
        while ptr:
            print(f"{ptr.val} -> ", end='')
            ptr = ptr.next
        print('None')


def mergeKLists(lists):
    head = dummy = ListNode(0)
    heap = []
    for l in lists:
        if l:
            heapq.heappush(heap, (Wrapper(l), l))

    while heap:
        _, node = heapq.heappop(heap)
        dummy.next = node
        dummy = dummy.next
        node = node.next
        if node:
            heapq.heappush(heap, (Wrapper(node), node))

    return head.next


l1 = ListGenerator()
l2 = ListGenerator()
l1.root = [1,3,4]
l2.root = [2,3,4]
ListGenerator.print_element(l1.root)
ListGenerator.print_element(l2.root)

merged_result = mergeKLists([l1.root, l2.root])
ListGenerator.print_element(merged_result)

1 -> 3 -> 4 -> None
2 -> 3 -> 4 -> None
1 -> 2 -> 3 -> 3 -> 4 -> 4 -> None


When doing a comparison in heapq, Python looks for __lt__() first. If it is not defined, it will look for __gt__(). If neither is defined, it throws TypeError: '<' not supported between instances of `class name` and `class name`. 

## Representing Classes
###  `__str__(self)`, `__repr__(self)`, `__format__(self)`
- The major difference between `str()` and `repr()` is intended audience. `repr()` is intended to produce output that is mostly machine-readable, whereas `str()` is intended to be human-readable.
- `format()` defines behavior for when an instance of the class is used in new-style string formatiing.

In [7]:
class Person:
    def __init__(self, fname, lname, age):
        self.__first_name = fname
        self.__last_name = lname
        self.__age = age

    @property
    def name(self):
        return f"{self.__first_name} {self.__last_name}"
    
    @name.setter
    def name(self, fname = '', lname= ''):
        self.__first_name = fname
        self.__last_name = lname
    
    @property
    def age(self):
        return self.__age

    @age.setter
    def age(self, age):
        self.__age = age

    def __repr__(self):
        return f"<Person object: {self.__first_name}>"
    
    def __str__(self):
        return self.name
    
    def __format__(self, fmt):
        if fmt == "%d":
            return f"{self.__age}-year-old"
        else:
            return f"{self.name}: {self.__age}-year-old"

john = Person("John", "Doe", 10)
print(john)
print(f"{john:%d}")
print(f"{john}")
john

John Doe
10-year-old
John Doe: 10-year-old


<Person object: John>

### Other magic methods for representing a class: 
- `__hash__(self)`, `__nonezero__(self)`, `__dir__(self)`, `__sizeof__(self)`

## Numeric magic methods
### Unary operators and functions
`__pos__(self)`, `__neg__(self)`, `__abs__(self)`, `__invert__(self)`, `__round__(self)`, `__floor__(self)`, `__ceil__(self)`, `__trunc__(self)`

### Normal arithmetic operators
`__add__(self, other)`, `__sub__(self, other)`, `__mul__(self, other)`, `__floordiv__(self, other)` -> `//`, `__div__(self, other)` -> `/`, `__truediv__(self, other)`, `__mod__(self, other)`, `__divmod__(self, other)`, `__pow__(self, other)` -> `**`, `__lshift__(self, other)`, `__rshift__(self, other)`, `__and__(self, other)`, `__or__(self, other)`, `__xor__(self, other)`

### Reflected arithmetic operators

### Augmented assignment ( +=, -=, /=, //=, %=, **= ...)

### Type conversion magic methods

## References
1. [Python - Class from TutorialsTeacher](https://www.tutorialsteacher.com/python/python-class)
2. [Python - Magic Methods from TutorialsTeacher](https://www.tutorialsteacher.com/python/magic-methods-in-python)
3. [wfitz / Python / Classes.ipynb](https://github.com/wfitz/Python/blob/master/Classes.ipynb)
4. [A Guide to Python's Magic Methods](https://rszalski.github.io/magicmethods/)